<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:140 px; height:140px;">
<img src="https://cloudrun.co/img/logo_noname.png" alt="Cloudrun Logo" style="height: 140px;">
</div>

<h1>Step 2: Pre-process Downloaded NCEI GFS+NAM Data from Step 1</h1>
<h2>By Kayla Besong</h2>
This program is specific to Pensecola Airport Meteorological data, generating netcdf files from previously downloaded GFS/NAM GRIB with the closest latitude/longitude point and variables with names that directly match those used in observation outputs. The resulting netcdfs will be read in step 3, where direct comparision of variables for both NCEP models, Cloudrun output, and observation will take place. Because this is specific to Pensecola Airport, this notebook would have to be adjusted only in the 'xr_retrun_pensecola' function to adjust to other locations and edit variables. If other variables are desried, the typeOfLevel and cfVarName will need to be added. A list of variables/level can be accessed by clicking on the .inv file in the url directory where the file was downloaded from in step 1. The variables will then need to be added to 'xr_retrun_pensecola' as an xarray variable and the ncep_vars dictionary input below. 


<div style="clear:both"></div>
</div>

<hr style="height:2px;">

In [2]:
import xarray as xr
import numpy as np
import os
import pandas as pd 
import matplotlib.dates as mdates
import warnings; warnings.simplefilter('ignore')


In [3]:
def neci_grab_variables(ncep_var_dict, directory):
    
    
    '''
    
    This function utilizes dictionaries and the ncei_grib_reading function to loop through each run and extract multiple variables with the type of level as the key and the variable as the value. 
    This is to iterate over all desired variables be stored in one dictionary for later processing into array based on run. The reasoning is that to open using cfgrib engine, the type of 
    level and variable needs to be specified and only one variable can be opened for a given run at a time. The approach is to consider a variable and open all the files/runs and store them 
    in a new dictionary. The resulting dictrionary will have the same keys, but the values will now be a list of xarrays with one variable with the length equal to the amount of runs. 
    
    '''
    
    
    new_dict = {}                                               # generate empty dictionary for given run 
    

    for key,value in ncep_var_dict.items():

        if type(value) == list:
            for i in range(len(value)):
                
                new_key = key + '_' + value[i]
                
                if value[i] == 'prate' or key == 'unknown':        # surface and unknown levels require stepType instant or avg 
                    
                    new_dict[key] = ncei_grib_reading(directory, key, value[i], stepType = 'instant')        # call ncei reading to loop through directory and store 
                
                elif value[i] == 'tp':
                    
                    new_dict[key] = ncei_grib_reading(directory, key, value[i], stepType = 'accum')        # call ncei reading to loop through directory and store 

                
                else:
                    
                    new_dict[new_key] = ncei_grib_reading(directory, key, value[i])

        else:
            
            if value == 'prate': 
                
                new_dict[key] = ncei_grib_reading(directory, key, value, stepType = 'instant')
            
            elif value == 'tp':
                
                new_dict[key] = ncei_grib_reading(directory, key, value, stepType = 'accum')

            else:
                
                new_dict[key] = ncei_grib_reading(directory, key, value)
        
            
    return new_dict


In [4]:
def ncei_grib_reading(directory, typeOfLevel, cfVarName, stepType = None):
    
    '''
    
    This function loops through the directories/subdirectories established in step 1 that conatin the downloaded GFS and NAM data. Each file in files loop takes each run, 
    opens the variables requested, then concatenates along the forecast period (72 for this example, total of 25 valid_times). 
    The result is an array of xrarry dataarrays that contain the runs with one variable.
    
    '''   

    
    collection_of_runs = []                                                                                               ## empty array to store inddividual runs 
    
    for root, dirs, files in sorted(os.walk(directory)):                                                                  ## navigate os

        print('reading ' + root + ' variable ' + typeOfLevel + ' '+ cfVarName)                                            ## ensure correct directory, status

        datas = []
        
        for file in sorted(files):


            if file[-4:] == 'grb2':                                           ## error handle, read only netcdf 
                
                
                ## the next set of conidtions were required in specifying levels of detail needed for xarray to handle the grib file. If too many or too few filter_by_keys inputs are present, the opening of the fle will not be achieved 
                
                
                if typeOfLevel == 'surface' and stepType == 'instant':
                    
                    datas.append(xr.open_dataset(root+'/'+file, engine = 'cfgrib',  backend_kwargs = dict(filter_by_keys={'typeOfLevel': typeOfLevel, 'cfVarName': cfVarName, 'stepType': stepType}))[cfVarName])                 
                                    
                elif typeOfLevel == 'surface' and stepType == 'accum':
                    
                    datas.append(xr.open_dataset(root+'/'+file, engine = 'cfgrib',  backend_kwargs = dict(filter_by_keys={'typeOfLevel': typeOfLevel, 'stepType': stepType}))[cfVarName])                 
                
                elif typeOfLevel == 'heightAboveGround':
                    
                    datas.append(xr.open_dataset(root+'/'+file, engine = 'cfgrib',  backend_kwargs = dict(filter_by_keys={'typeOfLevel': typeOfLevel, 'cfVarName': cfVarName}))[cfVarName])                 
                
                elif typeOfLevel == 'meanSea':

                    datas.append(xr.open_dataset(root+'/'+file, engine = 'cfgrib',  backend_kwargs = dict(filter_by_keys={'typeOfLevel': typeOfLevel}))[cfVarName])

                else:
                    print('has !!!!!!NOT!!!!! appended ' + root + ' variable ' + typeOfLevel + ' '+ cfVarName)
                    
                                                                                                                                     
            else:

                continue
        
        if len(datas) > 0:                                                                   ## make sure non empty list of data files are being appended
                        
            collection_of_runs.append(xr.concat(datas, dim = 'valid_time'))                  ## concatinatinate dataset for individual run set 

        else:
            continue
            
    return collection_of_runs
     

In [5]:
def xr_retrun_pensecola(new_dict, directory):
    
    
    '''
    
    This function takes the output of ncei_grib_reading, an array of xrarry dataarrays that contain all runs for one variable, and turns it into one dataset per run for all variables with unit conversions.
    This step also finds the nearest gridpoint from the model to the PENSACOLA AIRPORT specifically lat+lon point. These variables are also specific to PENSACOLA Airport station data
    variable names. This is so that when post processing, the same titles can be used to select all data i.e. this function homogenizes all model data into the same format as obs, units included. 
    The output is an array of xarrays with all variables for for a given run (verses a sweet of datararrays for all runs with one variable) to be exported as netcdf files. 
    
    '''  
    
    
    new_dict_keys = list(new_dict.keys())
    
    ## Pensecola Aitport gridpoint. Can become an input variable of this function later on for other locations. 

    lat = 30.47806
    lon = 360.-87.18694

    
    ## locating the neartest point to lat/lon above
    
    abslat = np.abs(new_dict[new_dict_keys[0]][0].latitude-lat)
    abslon = np.abs(new_dict[new_dict_keys[0]][0].longitude-lon)
    c = np.maximum(abslon, abslat)

    loc = np.where(c == np.min(c))

    
    ## the x and y cords between GFS and NAM seem to be opposite when the above procedure is used to determine nearest grid point
    ## hence the condition below to use as selection points for x-array datasets below.
    
    
    if directory[0:3] == 'GFS':
        
        x_sel = loc[0][0]               # 'true' x
        y_sel = loc[1][0]               # 'true' y
        
    elif directory[0:3] == 'NAM':
        
        x_sel = loc[1][0]               # 'true' y
        y_sel = loc[0][0]               # 'true' x
        


    file_lab = [i for i in sorted(os.listdir(directory)) if len(i) == 10]            # ensure directory == directory developed in Step 1 ~ ncei_data_grab.ipynb, the len of this directory = len of collection of runs = length needing to be looped through. Used as an error handling measure. 
    array_out = []

    
    
    ## loop through each collection of runs and extract each variable for that run, convert as needed, select nearest grid point to obs and convert 

    for j in range(len(file_lab)):
        

        wind_speed = np.sqrt(new_dict['heightAboveGround_u10'][j][:].isel(longitude = x_sel, latitude = y_sel)**2 + new_dict['heightAboveGround_v10'][j][:].isel(longitude = x_sel, latitude = y_sel)**2)
        wind_dir1 = np.arctan2(-new_dict['heightAboveGround_u10'][j][:].isel(longitude = x_sel, latitude = y_sel).values, -new_dict['heightAboveGround_v10'][j][:].isel(longitude = x_sel, latitude = y_sel))*(-180./np.pi)
        wind_dir1[np.where(wind_dir1 < 0)] += 360

        array_out.append(xr.Dataset({'wind_speed': (['time'], wind_speed),
                                   'wind_dir': (['time'], wind_dir1),
                                   'air_pressure_at_mean_sea_level': (['time'], new_dict['meanSea'][j][:].isel(longitude = x_sel, latitude = y_sel)/100.),
                                   'air_temperature_2m': (['time'], new_dict['heightAboveGround_t2m'][j][:].isel(longitude = x_sel, latitude = y_sel)-273.15),
                                   'relative_humidity_2m': (['time'], new_dict['heightAboveGround_r2'][j][:].isel(longitude = x_sel, latitude = y_sel)),
                                   'rainfall_amount': (['time'], new_dict['surface'][j][:].isel(longitude = x_sel, latitude = y_sel))},

                                    coords = {'time': sorted(new_dict['meanSea'][j][:].valid_time.values)}))


    return array_out


In [32]:
def out_to_nc(array_out, directory):
    
    
    '''
    
    This function takes the output of xr_retrun_pensecola, array xarray Datasets with all variables for for a given run and exports each run with all desired variables to a netcdf file. 
    
    ''' 
    
    file_lab = [i for i in sorted(os.listdir(directory)) if len(i) == 10]
    main_dir = 'processed_ncei'
    sub_dir = directory[0:3] 
   
    try: os.makedirs(os.path.join(main_dir,sub_dir))
    except OSError: print('Directory already exists for ' + main_dir + 'and ' + sub_dir)  ## let it be known if this directory for model + year_mon_day already exists
    
    paths = [main_dir + '/' + sub_dir + '/' + directory[0:3] +'_%s.nc' %  ymdt for ymdt in file_lab]
    
    for array, path in zip(array_out, paths):
        array.to_netcdf(path)
    

## GFS

In [6]:
ncep_var_gfs = {'meanSea': 'mslet', 'surface': 'prate', 'heightAboveGround': ['t2m', 'r2', 'u10', 'v10']}
ncep_var_gfs

{'meanSea': 'mslet',
 'surface': 'prate',
 'heightAboveGround': ['t2m', 'r2', 'u10', 'v10']}

In [ ]:
%%time 

out_to_nc(xr_retrun_pensecola(neci_grab_variables(ncep_var_gfs, "GFS_data"), 'GFS_data'), "GFS_data")

## NAM

In [16]:
ncep_var_nam = {'meanSea': 'mslet', 'surface': 'tp', 'heightAboveGround': ['t2m', 'r2', 'u10', 'v10']}
ncep_var_nam

{'meanSea': 'mslet',
 'surface': 'tp',
 'heightAboveGround': ['t2m', 'r2', 'u10', 'v10']}

In [ ]:
%%time 

out_to_nc(xr_retrun_pensecola(neci_grab_variables(ncep_var_nam, "NAM_data"), "NAM_data"), 'NAM_data')